In [ ]:
cd /content/drive/MyDrive/GenAI Project/

/content/drive/MyDrive/GenAI Project


In [ ]:
!pip install RapidFuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 22.7 MB/s eta 0:00:00


In [ ]:
import json

def read_json(filepath):
    """Reads a JSON file and returns the data."""
    with open(filepath, 'r') as f:
        return json.load(f)

def write_json(data, filepath):
    """Writes data to a JSON file."""
    with open(filepath, 'w') as f:
        json.dump(data, f, indent=4)

In [ ]:
entity_index = read_json('Dataset/Corpus/entity_index.json')
mahabharata_questions = read_json('Dataset/Validation/mahabharata_questions.json')
entities_kb = read_json('Dataset/Corpus/entities_kb.json')
chapter_entity_ids = read_json('Dataset/Corpus/chapter_entity_ids.json')


### Modules

In [ ]:
from rapidfuzz import fuzz, process
from collections import defaultdict

class ProcessQuery:

  def __init__(self, query, entity_index):
      self.query = query
      self.entity_index = entity_index

  def extract_query_entities(self, query_word, threshold=90):
      matches = process.extract(query_word, list(self.entity_index.keys()), scorer=fuzz.token_sort_ratio, limit=5)
      matched_entities = []
      scores = []
      for _match, score, _ in matches:
          if float(score) >= threshold:
              matched_entities.append(entity_index[_match])
              scores.append(score)
      return matched_entities, scores

  def score_query_entities(self):
    self.query_entities = {}
    for query_word in self.query.split():
      entities, scores = self.extract_query_entities(query_word.lower())
      if entities:
        for _e in entities:
          for e in _e:
            self.query_entities[e] = scores[0]


### Explore

In [ ]:
p = ProcessQuery("When did Bhishma kidnap Amba", entity_index)
p.score_query_entities()

In [ ]:
p.query_entities

{'e1497': 100.0, 'e1498': 100.0, 'e1499': 100.0, 'e413': 100.0}

In [ ]:
data = []

for gt, v in mahabharata_questions.items():
  for q in v['questions']:
    p = ProcessQuery(q, entity_index)
    p.score_query_entities()
    entities_names = [entities_kb[k] for k in p.query_entities.keys()]
    entities_ids = list(p.query_entities.keys())
    scores = list(p.query_entities.values())
    data.append({
          "ground_truth": gt,
          "question": q,
          "entity_ids": entities_ids,
          "entity_scores": scores
    })


In [ ]:
import pandas as pd
df = pd.DataFrame(data)

In [ ]:
df

,ground_truth,question,entity_ids,entity_scores
0,1.1,What is the main premise and scope of the Maha...,[e6721],[100.0]
1,1.1,Which topics are briefly listed or summarized ...,[e6721],[95.65217391304348]
2,1.1,What glories or benefits are said to come from...,[e6721],[95.65217391304348]
3,1.2,What is the significance of the Samantpanchak ...,[e6721],[95.65217391304348]
4,1.2,How is the Akshauhini army described or enumer...,"[e376, e6721]","[100.0, 95.65217391304348]"
...,...,...,...,...
446,3.312,Where do the Pandavas fall unconscious one aft...,"[e7969, e7970, e7971, e7972, e7973, e7974, e79...","[93.33333333333333, 93.33333333333333, 93.3333..."
447,3.312,Which section precedes Yudhishthira’s encounte...,"[e12633, e12536, e12537, e12538]","[92.3076923076923, 92.3076923076923, 92.307692..."
448,3.313,Which chapter contains the dialogue between Ya...,"[e12536, e12537, e12538, e370, e371, e12633, e...","[100.0, 100.0, 100.0, 100.0, 100.0, 96.0, 96.0..."
449,3.313,Where does Yaksha restore the lives of the Pan...,"[e12536, e12537, e12538, e370, e371, e7969, e7...","[100.0, 100.0, 100.0, 100.0, 100.0, 93.3333333..."


### Scores

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity

def chapter_similarity_propagation(chapter_entities, query_entities, query_weights, k=5, alpha=2.0):
    """
    Compute chapter relevance given entity frequencies and query entities.
    Stage 1: Direct relevance using TF-IDF
    Stage 2: Propagated relevance via similarity graph

    Args:
        chapter_entities: dict {chapter_id: {entity: frequency, ...}}
        query_entities: list of entity names (IDs)
        query_weights: list/array of weights for each query entity
        k: number of top chapters to use for propagation
        alpha: sharpness factor for sigmoid normalization

    Returns:
        top3_ids: list of top 3 chapter IDs
        all_scores: dict {chapter_id: normalized_score}
    """

    # ---------- Stage 0: Build entity matrix ----------
    all_entities = sorted({e for ents in chapter_entities.values() for e in ents})
    df = pd.DataFrame(0, index=chapter_entities.keys(), columns=all_entities)
    for ch, ents in chapter_entities.items():
        for ent, freq in ents.items():
            df.loc[ch, ent] = freq

    # Normalize term frequencies per chapter
    df_norm = df.div(df.sum(axis=1), axis=0).fillna(0)

    # ---------- Stage 1: TF-IDF ----------
    tfidf = TfidfTransformer()
    X_tfidf = tfidf.fit_transform(df_norm.values)

    # ---------- Stage 1.5: Match query entities safely ----------
    valid_query_pairs = [
        (all_entities.index(e), w)
        for e, w in zip(query_entities, query_weights)
        if e in all_entities
    ]

    if not valid_query_pairs:
        # No matching entities found — return zero scores
        zero_scores = pd.Series(0.0, index=df.index)
        return [], zero_scores.to_dict()

    query_indices, valid_weights = zip(*valid_query_pairs)
    query_indices = list(query_indices)
    query_weights = np.array(valid_weights)

    # ---------- Stage 2: Direct relevance ----------
    relevance = X_tfidf[:, query_indices].dot(query_weights)
    if not isinstance(relevance, np.ndarray):
        relevance = relevance.toarray()
    relevance = np.ravel(relevance)

    # ---------- Stage 3: Propagate via similarity ----------
    topk_idx = np.argsort(-relevance)[:k]
    sim_matrix = cosine_similarity(X_tfidf)
    prop_scores = np.zeros(len(df))
    chapter_has_query_entity = (df[query_entities].sum(axis=1) > 0).astype(int).values
    for idx in topk_idx:
        # 🔸 Only propagate to masked chapters
        masked_sim = sim_matrix[:, idx] * chapter_has_query_entity
        prop_scores += relevance[idx] * masked_sim


    # ---------- Stage 4: Normalize ----------
    scores_normalized = 1 / (1 + np.exp(-alpha * (prop_scores - np.mean(prop_scores))))

    # ---------- Stage 5: Output ----------
    final_scores = pd.Series(scores_normalized, index=df.index).sort_values(ascending=False)
    top3_ids = list(final_scores.index[:3])
    all_scores = final_scores.to_dict()

    return top3_ids, all_scores


In [ ]:
top_chapters_list = []
final_data = {}
from tqdm import tqdm

for index, row in tqdm(df.iterrows(), total = df.shape[0]):
    query_entities = row['entity_ids']
    query_weights = row['entity_scores']
    try:
        topk, final_scores = chapter_similarity_propagation(chapter_entity_ids, query_entities, query_weights)
        top_chapters_list.append(topk)
        final_data[index] = final_scores
    except ValueError as e:
        print(f"Skipping row {index} due to error: {e}")
        final_data[index] = {}
        top_chapters_list.append(None)


100%|██████████| 451/451 [23:37<00:00,  3.14s/it]


In [ ]:
df['top_chapters'] = top_chapters_list


In [ ]:
df

,ground_truth,question,entity_ids,entity_scores,top_chapters
0,1.1,What is the main premise and scope of the Maha...,[e6721],[100.0],"[1.13, 18.6, 1.1]"
1,1.1,Which topics are briefly listed or summarized ...,[e6721],[95.65217391304348],"[1.13, 18.6, 1.1]"
2,1.1,What glories or benefits are said to come from...,[e6721],[95.65217391304348],"[1.13, 18.6, 1.1]"
3,1.2,What is the significance of the Samantpanchak ...,[e6721],[95.65217391304348],"[1.13, 18.6, 1.1]"
4,1.2,How is the Akshauhini army described or enumer...,"[e376, e6721]","[100.0, 95.65217391304348]","[18.1, 15.27, 7.151]"
...,...,...,...,...,...
446,3.312,Where do the Pandavas fall unconscious one aft...,"[e7969, e7970, e7971, e7972, e7973, e7974, e79...","[93.33333333333333, 93.33333333333333, 93.3333...","[15.38, 9.13, 9.15]"
447,3.312,Which section precedes Yudhishthira’s encounte...,"[e12633, e12536, e12537, e12538]","[92.3076923076923, 92.3076923076923, 92.307692...","[15.36, 18.4, 15.37]"
448,3.313,Which chapter contains the dialogue between Ya...,"[e12536, e12537, e12538, e370, e371, e12633, e...","[100.0, 100.0, 100.0, 100.0, 100.0, 96.0, 96.0...","[15.36, 15.37, 15.39]"
449,3.313,Where does Yaksha restore the lives of the Pan...,"[e12536, e12537, e12538, e370, e371, e7969, e7...","[100.0, 100.0, 100.0, 100.0, 100.0, 93.3333333...","[1.2, 15.36, 15.39]"


In [ ]:
df.to_excel('Retrieval Experiments/1.7/df.xlsx')

In [ ]:
with open('Retrieval Experiments/1.7/graph_scores.json', 'w') as f:
    json.dump(final_data, f, indent = 4)

In [ ]:
final_data = read_json('Retrieval Experiments/1.7/graph_scores.json')

In [ ]:
query_scores = final_data['10']
sorted(query_scores.items(), key=lambda item: item[1], reverse=True)


[('15.35', 1.0),
 ('15.34', 1.0),
 ('1.50', 1.0),
 ('1.51', 1.0),
 ('1.37', 1.0),
 ('1.49', 1.0),
 ('3.203', 1.0),
 ('3.204', 1.0),
 ('4.62', 1.0),
 ('1.60', 1.0),
 ('14.53', 1.0),
 ('1.64', 1.0),
 ('1.53', 1.0),
 ('1.58', 1.0),
 ('1.56', 1.0),
 ('1.55', 1.0),
 ('3.185', 1.0),
 ('12.266', 1.0),
 ('3.202', 1.0),
 ('3.201', 1.0),
 ('1.110', 1.0),
 ('1.116', 1.0),
 ('14.59', 1.0),
 ('14.58', 1.0),
 ('14.57', 1.0),
 ('14.56', 1.0),
 ('14.54', 1.0),
 ('14.55', 1.0),
 ('13.102', 1.0),
 ('13.78', 1.0),
 ('3.50', 1.0),
 ('14.90', 1.0),
 ('1.158', 1.0),
 ('1.159', 1.0),
 ('12.168', 1.0),
 ('14.91', 1.0),
 ('12.150', 1.0),
 ('12.151', 1.0),
 ('12.152', 1.0),
 ('12.129', 1.0),
 ('5.155', 1.0),
 ('14.71', 1.0),
 ('14.70', 1.0),
 ('12.169', 1.0),
 ('12.170', 1.0),
 ('12.171', 1.0),
 ('12.172', 1.0),
 ('12.173', 1.0),
 ('1.3', 1.0),
 ('2.60', 1.0),
 ('2.40', 1.0),
 ('1.215', 1.0),
 ('1.20', 1.0),
 ('1.228', 1.0),
 ('1.222', 1.0),
 ('2.21', 1.0),
 ('1.15', 1.0),
 ('1.214', 0.9999999999999998),
 ('1.1

In [ ]:
chapter_entity_ids['3.152']

{'e11327': 1, 'e1454': 3, 'e3964': 1, 'e8964': 1, 'e4777': 1, 'e3660': 1}

In [ ]:
query_entities = ['e11250', 'e4777', 'e4778', 'e4779', 'e4780', 'e4781', 'e4782', 'e4783', 'e4784', 'e4785', 'e4786', 'e4787']
query_weights = [93.33333333333333, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]
chapter_similarity_propagation(chapter_entity_ids, query_entities, query_weights)

(['3.152', '12.150', '12.151'],
 {'3.152': 1.0,
  '12.150': 1.0,
  '12.151': 1.0,
  '2.13': 1.0,
  '14.59': 1.0,
  '2.21': 1.0,
  '4.62': 1.0,
  '1.68': 1.0,
  '13.167': 1.0,
  '14.55': 1.0,
  '5.84': 1.0,
  '14.63': 1.0,
  '1.60': 1.0,
  '1.58': 1.0,
  '12.152': 1.0,
  '1.56': 1.0,
  '1.55': 1.0,
  '1.54': 1.0,
  '1.53': 1.0,
  '1.51': 1.0,
  '1.50': 1.0,
  '1.49': 1.0,
  '1.64': 1.0,
  '14.54': 1.0,
  '14.53': 1.0,
  '1.128': 1.0,
  '3.201': 1.0,
  '8.1': 1.0,
  '3.202': 1.0,
  '3.203': 1.0,
  '1.116': 1.0,
  '3.204': 1.0,
  '14.85': 1.0,
  '3.143': 1.0,
  '14.58': 1.0,
  '1.107': 1.0,
  '14.90': 1.0,
  '14.91': 1.0,
  '14.56': 1.0,
  '5.155': 1.0,
  '1.214': 1.0,
  '1.215': 1.0,
  '1.218': 1.0,
  '1.222': 1.0,
  '1.228': 1.0,
  '14.57': 1.0,
  '1.110': 1.0,
  '9.39': 1.0,
  '12.173': 1.0,
  '1.158': 1.0,
  '6.13': 1.0,
  '1.15': 1.0,
  '12.45': 1.0,
  '12.47': 1.0,
  '2.60': 1.0,
  '18.5': 1.0,
  '13.166': 1.0,
  '15.34': 1.0,
  '14.71': 1.0,
  '12.348': 1.0,
  '1.20': 1.0,
  '3.244

### Scores - 2

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity


def build_entity_matrix(ch_entities):
    """Create normalized entity frequency matrix (chapters × entities)."""
    all_entities = sorted({e for ents in ch_entities.values() for e in ents})
    df = pd.DataFrame(0, index=ch_entities.keys(), columns=all_entities)

    for ch, ents in ch_entities.items():
        for ent, freq in ents.items():
            df.loc[ch, ent] = freq

    # Normalize per chapter (term frequency normalization)
    df_norm = df.div(df.sum(axis=1), axis=0).fillna(0)
    return df, df_norm, all_entities

df_entities, df_norm, all_entities = build_entity_matrix(chapter_entity_ids)

In [ ]:
def compute_tfidf(df_norm):
    """Compute TF-IDF matrix from normalized entity frequencies."""
    tfidf = TfidfTransformer(norm=None)
    X_tfidf = tfidf.fit_transform(df_norm.values)
    return X_tfidf


def get_valid_query_indices(query_entities, query_weights, all_entities):
    """Return indices and weights of query entities that exist in the corpus."""
    valid_pairs = [
        (all_entities.index(e), w)
        for e, w in zip(query_entities, query_weights)
        if e in all_entities
    ]
    if not valid_pairs:
        return None, None
    query_indices, valid_weights = zip(*valid_pairs)
    return list(query_indices), np.array(valid_weights)

def group_consecutive_entities(entities):
    groups = []
    current_group = [entities[0]]

    for prev, curr in zip(entities, entities[1:]):
        if int(curr[1:]) == int(prev[1:]) + 1:  # check consecutive numbers
            current_group.append(curr)
        else:
            groups.append(current_group)
            current_group = [curr]
    groups.append(current_group)
    return groups

def compute_group_mask(df_entities, groups):
    """
    Return a binary mask [num_chapters x num_groups]
    where mask[i, g] = 1 if chapter i contains any entity from group g.
    """
    mask = np.zeros((len(df_entities), len(groups)), dtype=int)
    for g_idx, group in enumerate(groups):
        valid_entities = [e for e in group if e in df_entities.columns]
        mask[:, g_idx] = (df_entities[valid_entities].sum(axis=1) > 0).astype(int)
    return mask


def compute_direct_relevance(X_tfidf, query_indices, query_weights):
    """Compute direct relevance of each chapter using weighted query entities."""
    # Step 1: compute normal relevance
    relevance = np.ravel(X_tfidf[:, query_indices].dot(query_weights))

    # Step 2: build group mask
    groups = group_consecutive_entities(query_entities)
    group_mask = compute_group_mask(df_entities, groups)

    # Step 3: find which chapters have all groups represented
    has_all_groups = (group_mask.sum(axis=1) == len(groups))

    # Step 4: zero out relevance for chapters missing a group
    group_coverage = group_mask.sum(axis=1) / len(groups)
    relevance = relevance * group_coverage

    if not isinstance(relevance, np.ndarray):
        relevance = relevance.toarray()
    return np.ravel(relevance)


def propagate_relevance(X_tfidf, df_entities, query_entities, relevance, k=5):
    """Propagate relevance scores through similarity graph (masked)."""
    sim_matrix = cosine_similarity(X_tfidf)
    topk_idx = np.argsort(-relevance)[:k]
    prop_scores = np.zeros(len(df_entities))

    # Mask: chapters that contain at least one query entity
    valid_cols = [e for e in query_entities if e in df_entities.columns]
    chapter_mask = (df_entities[valid_cols].sum(axis=1) > 0).astype(int).values

    for idx in topk_idx:
        masked_sim = sim_matrix[:, idx] * chapter_mask
        prop_scores += relevance[idx] * masked_sim

    return prop_scores


def normalize_scores(prop_scores, alpha=2.0):
    """Apply sigmoid normalization to propagated scores."""
    return 1 / (1 + np.exp(-alpha * (prop_scores - np.mean(prop_scores))))

def chapter_similarity_propagation(chapter_entities, query_entities, query_weights, k=5, alpha=2.0):
    """
    Compute chapter relevance given entity frequencies and query entities.
    Stage 1: Direct relevance using TF-IDF
    Stage 2: Propagated relevance via similarity graph (masked)
    """

    # Stage 1: TF-IDF
    X_tfidf = compute_tfidf(df_norm)

    # Stage 1.5: Query entity matching
    query_indices, query_weights = get_valid_query_indices(query_entities, query_weights, all_entities)
    # print(query_indices, query_weights)
    if query_indices is None:
        zero_scores = pd.Series(0.0, index=df_entities.index)
        return [], zero_scores.to_dict()

    # Stage 2: Direct relevance
    relevance = compute_direct_relevance(X_tfidf, query_indices, query_weights)


    # Stage 3: Propagation
    prop_scores = propagate_relevance(X_tfidf, df_entities, query_entities, relevance, k=k)

    # Stage 4: Normalization
    scores_normalized = normalize_scores(prop_scores, alpha=alpha)

    # Stage 5: Output
    final_scores = pd.Series(scores_normalized, index=df_entities.index).sort_values(ascending=False)
    top3_ids = list(final_scores.index[:3])
    return top3_ids, final_scores.to_dict()


In [ ]:
top_chapters_list = []
final_data = {}
from tqdm import tqdm

for index, row in tqdm(df.iterrows(), total = df.shape[0]):
    query_entities = row['entity_ids']
    query_weights = row['entity_scores']
    try:
        topk, final_scores = chapter_similarity_propagation(chapter_entity_ids, query_entities, query_weights)
        top_chapters_list.append(topk)
        final_data[index] = final_scores
    except ValueError as e:
        print(f"Skipping row {index} due to error: {e}")
        final_data[index] = {}
        top_chapters_list.append(None)


NameError: name 'df' is not defined

In [ ]:
df

,ground_truth,question,entity_ids,entity_scores,top_chapters
0,1.1,What is the main premise and scope of the Maha...,[e6721],[100.0],"[13.76, 13.127, 1.62]"
1,1.1,Which topics are briefly listed or summarized ...,[e6721],[95.65217391304348],"[13.76, 13.127, 1.62]"
2,1.1,What glories or benefits are said to come from...,[e6721],[95.65217391304348],"[13.76, 13.127, 1.62]"
3,1.2,What is the significance of the Samantpanchak ...,[e6721],[95.65217391304348],"[13.76, 13.127, 1.62]"
4,1.2,How is the Akshauhini army described or enumer...,"[e376, e6721]","[100.0, 95.65217391304348]","[13.76, 1.59, 1.62]"
...,...,...,...,...,...
446,3.312,Where do the Pandavas fall unconscious one aft...,"[e7969, e7970, e7971, e7972, e7973, e7974, e79...","[93.33333333333333, 93.33333333333333, 93.3333...","[5.127, 5.128, 5.129]"
447,3.312,Which section precedes Yudhishthira’s encounte...,"[e12633, e12536, e12537, e12538]","[92.3076923076923, 92.3076923076923, 92.307692...","[7.12, 7.16, 7.17]"
448,3.313,Which chapter contains the dialogue between Ya...,"[e12536, e12537, e12538, e370, e371, e12633, e...","[100.0, 100.0, 100.0, 100.0, 100.0, 96.0, 96.0...","[15.36, 15.39, 7.111]"
449,3.313,Where does Yaksha restore the lives of the Pan...,"[e12536, e12537, e12538, e370, e371, e7969, e7...","[100.0, 100.0, 100.0, 100.0, 100.0, 93.3333333...","[9.56, 9.55, 17.3]"


In [ ]:
df['top_chapters'] = top_chapters_list
df.to_excel('Retrieval Experiments/1.7/df.xlsx')

In [ ]:
graph_scores = {}
for index, row in tqdm(df.iterrows(), total = df.shape[0]):
    graph_scores[row['question']] = final_data[index]

100%|██████████| 451/451 [00:00<00:00, 21477.99it/s]


In [ ]:
with open('Retrieval Experiments/1.7/graph_scores.json', 'w') as f:
    json.dump(graph_scores, f, indent = 4)

### Combined

In [ ]:
embedding_scores = read_json('Retrieval Experiments/1.9/embedding_scores_1.json')
graph_scores = read_json('Retrieval Experiments/1.7/graph_scores.json')

In [ ]:
len(embedding_scores), len(graph_scores)

(450, 450)

In [ ]:
combined_scores = {}

a = 0.9
b = 0.1

for question, scores in embedding_scores.items():
    combined_scores[question] = {}
    for chapter, score in scores.items():
        if chapter in graph_scores[question]:
            combined_scores[question][chapter] = (a * score) + (b * graph_scores[question][chapter])
        else:
            combined_scores[question][chapter] = score # Handle cases where a chapter is not in graph_scores

In [ ]:
top_5_chapters_combined = {}

chapters_ids_net = []
accuracies = []
for i, (question, scores) in enumerate(combined_scores.items()):
    sorted_chapters = sorted(scores.items(), key=lambda item: item[1], reverse=True)[:7]
    top_5_chapters_combined[question] = dict(sorted_chapters)
    cids = [float(chapter) for chapter, _ in sorted_chapters]
    chapters_ids_net.append(cids)
    gt = df.loc[df['question'] == question, 'ground_truth'].iloc[0]
    acc = 1 if float(gt) in cids else 0
    # print(gt, cids, acc)
    accuracies.append(acc)


In [ ]:
sum(accuracies)  ## Finetuned MPNET

242

In [ ]:
sum(accuracies)  ## Finetuned MPNET

192

In [ ]:
sum(accuracies)  ## Just MPNET

261

In [1]:
def get_combined_scores(a,b):
  combined_scores = {}
  for question, scores in embedding_scores.items():
      combined_scores[question] = {}
      for chapter, score in scores.items():
          if chapter in graph_scores[question]:
              combined_scores[question][chapter] = (a * score) + (b * graph_scores[question][chapter])
          else:
              combined_scores[question][chapter] = score # Handle cases where a chapter is not in graph_scores

  return combined_scores

def get_acc(a, b):

  combined_scores = get_combined_scores(a,b)
  top_5_chapters_combined = {}

  chapters_ids_net = []
  accuracies = []
  for i, (question, scores) in enumerate(combined_scores.items()):
      sorted_chapters = sorted(scores.items(), key=lambda item: item[1], reverse=True)[:5]
      top_5_chapters_combined[question] = dict(sorted_chapters)
      cids = [float(chapter) for chapter, _ in sorted_chapters]
      chapters_ids_net.append(cids)
      gt = df.loc[df['question'] == question, 'ground_truth'].iloc[0]
      acc = 1 if float(gt) in cids else 0
      # print(gt, cids, acc)
      accuracies.append(acc)

  return sum(accuracies) / len(accuracies)


In [2]:
A = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

for a in A:
  print(a, 1-a, get_acc(a, 1-a))

NameError: name 'embedding_scores' is not defined

In [ ]:
df_top_chapters_combined = pd.DataFrame()
df_top_chapters_combined['questions'] = top_5_chapters_combined.keys()
df_top_chapters_combined['top_chapters'] = top_5_chapters_combined.values()
df_top_chapters_combined['ground_truth'] = df['ground_truth'].values
df_top_chapters_combined['Accuracy'] = accuracies

In [ ]:
df_top_chapters_combined

,questions,top_chapters,ground_truth,Accuracy
0,What is the main premise and scope of the Maha...,"{'1.59': 0.6342726707458497, '1.62': 0.6289916...",1.1,1
1,Which topics are briefly listed or summarized ...,"{'1.59': 0.6004682064056397, '1.63': 0.5684526...",1.1,0
2,What glories or benefits are said to come from...,"{'18.5': 0.6810595035552979, '1.62': 0.6752132...",1.2,0
3,What is the significance of the Samantpanchak ...,"{'1.2': 0.5292163372039795, '1.63': 0.51986472...",1.2,1
4,How is the Akshauhini army described or enumer...,"{'5.155': 0.6352143282799219, '1.2': 0.5823237...",1.2,1
...,...,...,...,...
445,Where do the Pandavas fall unconscious one aft...,"{'1.128': 0.6509760856628418, '1.151': 0.64362...",3.312,0
446,Which section precedes Yudhishthira’s encounte...,"{'3.23': 0.6807969093322754, '3.156': 0.675207...",3.312,0
447,Which chapter contains the dialogue between Ya...,"{'3.314': 0.6760963916778564, '2.59': 0.656726...",3.313,0
448,Where does Yaksha restore the lives of the Pan...,"{'3.162': 0.7018239974975586, '3.314': 0.69297...",3.313,0


In [ ]:
233 / 450

0.5177777777777778